In [1]:
import numpy as np
import timeit
start = timeit.default_timer()
from datetime import datetime, date, timedelta
import pandas as pd
import yahoo_fin.stock_info as yf
import random

from stage import getStage

In [ ]:
nasdaqList = yf.tickers_nasdaq()
train, test = train_test_split(nasdaqList, test_size=0.3, random_state=0, shuffle=True)
trainSet1, trainSet2, trainSet3  = np.array_split(train,3)
trainSets = [trainSet1, trainSet2]

In [2]:
SectorDict = {
    "Energy":"XLE","Technology":"XLK",
    "Communication Services":"XLC",
    "Consumer Staples":"XLP",
    "Health Care":"XLV",
    "Consumer Cyclical":"XLY",
    "Industrial":"XLI",
    "Utilities":"XLU",
    "Materials":"XLB",
    "Financial":"XLF",
    "Real Estate":"XLRE"}
sectorDfList = []
for sector in SectorDict.values():
    sectorDfList.append(pd.read_pickle("stockData/sectorCharts/"+sector+".pkl"))

nyseDf = pd.read_html('stockList.html')
nyseDf = pd.DataFrame(nyseDf[0][0])
nyseList = []
for row in nyseDf.iterrows():
    ticker = str(row[1]).split()[1]
    if ticker == "Symbol":
        continue
    nyseList.append(ticker)
allList = []
nasdaqList = yf.tickers_nasdaq()
allList = set(nasdaqList + nyseList)

def baseTest(individual):
    now = datetime.now()
    now = now.strftime("%Y-%m-%d-%H:%M")
    print(individual[0])
    winningTrade = 0
    totalTrade = 0
    transaction = pd.read_pickle("transactionTemplate.pkl")
    transaction['holding'] = np.empty((len(transaction), 0)).tolist()
    goodSectorDf = sectorDfList[0].drop(['close','ticker','MA30','MA30Slope','RS'],axis=1)
    goodSectorDf['Sectors'] = [[] for _ in range(len(goodSectorDf))]
    goodSectorDf.index = sectorDfList[0].index
    for date in sectorDfList[0].index:
        listOfSector = []
        for df in sectorDfList:
            try:
                if df.at[date,'MA30Slope'] > individual[0][9] and df.loc[date,'RS'] > individual[0][10]:
                    if df.at[date,'close'] > df.at[date,'MA30']:
                        listOfSector.append(df.at[date,'ticker'])
            except: 
                pass
        goodSectorDf.at[date,'Sectors'] = set(listOfSector)
    for symbol in random.sample(nyseList,1000):
        df = getStage(symbol,individual[0],goodSectorDf)
        if df.empty:
            continue
        drift = -1
        for index, element in df.iterrows():
            if drift == -1:
                drift = index.weekday()
            index = index - timedelta(days= drift)
            index = index.strftime('%Y-%m-%d')
            rowNumber = df.index.get_loc(index)
            # 1 => 'holding'
            transaction.iat[rowNumber][1].append((symbol,element.close,0))
    total = 100
    prevIndex = ''
    first = True
    transactionCopy = transaction
    transactionCopy['total'] = 0
    for index, element in transactionCopy.iterrows():
        index = index.strftime('%Y-%m-%d')
        if len(element.holding) == 0:
            transactionCopy['total'][index] = total
            continue
        if first:
            first = False
            transactionCopy['total'][index] = total
        else:
            prevData = transactionCopy.loc[prevIndex].holding
            if len(prevData):
                total = 0
                removeList = []
                close = 0
                for i in range(len(prevData)):
                    for j in range(len(element.holding)):
                        if element.holding[j][0] == prevData[i][0]:
                            close = element.holding[j][1]
                            if element.holding[j][2] == -1:
                                removeList.append(element.holding[j])
                            break
                    total += close*prevData[i][2]
                transactionCopy['total'][index] = total
            else:
                transactionCopy['total'][index] = transactionCopy.loc[prevIndex].total
        if len(element.holding):
            allocation = total/len(element.holding)
        for i in range(len(element.holding)):
            element.holding[i] = (element.holding[i][0],element.holding[i][1],allocation/element.holding[i][1])
        prevIndex = index
    stockHolding  = 0
    for i in transactionCopy.iterrows():
        stockHolding += len(i[1]['holding'])
    print(stockHolding/(transactionCopy.shape[0]))
    print(transactionCopy.iloc[-1]['total'])
    if totalTrade==0:
        print(winningTrade, totalTrade, 0)
    else:
        print(winningTrade, totalTrade, winningTrade/totalTrade)
    if stockHolding/(transactionCopy.shape[0])<8:
        return 0,
    resultFile = open("resultML1.txt","a")
    resultFile.write(str(individual[0])+"\n")
    resultFile.write(str(stockHolding/(transactionCopy.shape[0]))+"\n")
    resultFile.write(str(transactionCopy.iloc[-1]['total'])+"\n")
    resultFile.write(str(winningTrade)+" "+str(totalTrade)+" "+str(winningTrade/totalTrade)+"\n")
    resultFile.close()
    return transactionCopy.iloc[-1]['total'],